# Import part

In [3]:
import sys
sys.path.append('/content/HW5/')

In [4]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 24.1 MB/s eta 0:00:00


In [7]:
import molecular_descriptors
from utils import descriptor_target_split, descriptor_target_join
from rdkit import Chem
import pandas as pd

In [71]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer as Imputer
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
import urllib.request
import json
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.svm import LinearSVR
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction

In [127]:
def fit_Ridge(X_train, X_test, y_train, y_test):
    a = Imputer(strategy='median')
    b = StandardScaler()
    c = SelectKBest(score_func=mutual_info_regression)
    clf = RidgeCV(alphas=[1e-2, 1, 5, 10, 20, 30], cv=10)
    model = Pipeline([('impute', a), ('scaling', b), ('anova', c), ('ridge', clf)])

    parameters = {
        'anova__k': [5, 10, 20, 40],
        'ridge__alphas': [[1e-2], [1], [5], [10], [20], [30]]  # List of lists for alphas
    }

    grid = GridSearchCV(model, parameters)
    grid.fit(X_train, y_train)
    y_pred = grid.predict(X_test)

    metric = [grid.score(X_test, y_test),
              metrics.explained_variance_score(y_test, y_pred),
              metrics.mean_absolute_error(y_test, y_pred),
              metrics.mean_squared_error(y_test, y_pred),
              metrics.median_absolute_error(y_test, y_pred),
              metrics.r2_score(y_test, y_pred)]

    best_k = grid.best_params_['anova__k']
    best_alpha = grid.best_params_['ridge__alphas'][0]

    print(f"Best parameters are: anova_k: {best_k}, alpha: {best_alpha}")

    return grid, y_pred, metric

In [81]:
def desc_calc(data, mode):
    return molecular_descriptors.getAllDescriptors(data, mode)

In [126]:
def sar_model_evaluation(descriptors: pd.DataFrame):
    y = descriptors['Target']
    X = descriptors.drop(columns=['Target'])
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    model1, y_pred1, metrics1 = fit_Ridge(X_train, X_test, y_train, y_test)
    return model1, y_pred1, metrics1

In [125]:
def sar_model_train(descriptors_train: pd.DataFrame, indices):
    y_train = descriptors_train['Target']
    X_train = descriptors_train.drop(columns=['Target'])
    X_train = X_train[X_train.columns[indices]]
    a = Imputer(missing_values=np.nan, strategy='median')
    b = StandardScaler()
    clf = RidgeCV(cv=10, alphas=[30]) #based on print(f"Best parameters are: anova_k: {best_k}, alpha: {best_alpha}")
    #line my best parameters are anova_k: 40, alpha: 30
    model = Pipeline([('impute', a), ('scaling', b), ('rf', clf)])
    model.fit(X_train, y_train)
    return model

In [124]:
def sar_model_predict(model, descriptors_pred, indices):
    X_pred = descriptors_pred
    X_pred = X_pred[X_pred.columns[indices]]
    return model.predict(X_pred)

# PubChem

In [123]:
def pubchem_parsing(url):
    req = urllib.request.Request(url)
    res = urllib.request.urlopen(req).read()
    fin = json.loads(res.decode())
    return fin


def get_similar_cids(compound_smiles, threshold=95, maxentries=10):
    pubchem_pug_rest_api_link = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/"
    pubchem_pug_rest_api_link+= "fastsimilarity_2d/smiles/%(smiles)s/cids/JSON?Threshold=%(threshold)s&MaxRecords=%(maxentries)s" % {
        "smiles": compound_smiles, "threshold": threshold, "maxentries": maxentries}
    similar_cids = pubchem_parsing(pubchem_pug_rest_api_link)['IdentifierList']['CID']
    return similar_cids


def get_xlogp(compound_cid):
    pubchem_pug_rest_api_link = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/"
    pubchem_pug_rest_api_link += "compound/cid/%s/property/XLogP/JSON" % compound_cid
    try:
        xlogp = pubchem_parsing(pubchem_pug_rest_api_link)['PropertyTable']['Properties'][0]['XLogP']
        return xlogp
    except KeyError:
        return None

# Main part

In [132]:
if __name__ == "__main__":

    pd.set_option.use_inf_as_na = True

    # loading data
    train_data = pd.read_csv('/content/HW5/logp_100.csv')
    pred_data = pd.read_csv('/content/HW5/logp_inputs.csv')
    cpds = [row for row in pred_data.loc[:, 'SMILES']]

    # calculating descriptors
    print("Calculating descriptors for training data...")
    train_descriptors = desc_calc(train_data, mode='train')
    print("Calculating descriptors for prediction data...")
    pred_descriptors = desc_calc(pred_data, mode='pred')

    # finding best estimator
    print("Evaluating regression model parameters...")
    model = sar_model_evaluation(train_descriptors)
    #print('Best parameters are:', model[0].best_params_, best_alpha)
    cols = model[0].best_estimator_.named_steps['anova'].get_support(indices=True)

    # train the best estimator and predict values
    print("Training the model with the best parameters...")
    final_model = sar_model_train(train_descriptors, cols)

for cpd in cpds:
        cpd_descriptors = pred_descriptors[pred_descriptors['SMILES']==cpd]
        pred = sar_model_predict(final_model, descriptors_pred = cpd_descriptors, indices = cols)
        print(f"Predicted LogP value for compound {cpd}:", pred)

        result = []

        print("Searching for similar compunds...")
        similarity = get_similar_cids(compound_smiles = cpd) # related pubchem function

        print("Filtering logP...")
        for cid in similarity:
            xlogp = get_xlogp(cid) # related pubchem function
            if xlogp:
                if xlogp <= pred*1.1 and xlogp >=pred*0.9:
                    result.append((cid, xlogp))

        print(f"Request for compound {cpd} completed. I found the following CIDs in PubChem with XLogP in the range of {pred}+- 10%: {result}")

Calculating descriptors for training data...
Calculating Molecular Descriptors Completed.
Calculating descriptors for prediction data...
Calculating Molecular Descriptors Completed.
Evaluating regression model parameters...
Best parameters are: anova_k: 40, alpha: 30
Training the model with the best parameters...
Predicted LogP value for compound CC1=CC2=C(C=C1C)NC(=O)CC2C3=CC=CC=C3OC: [4.3486695]
Searching for similar compunds...
Filtering logP...
Request for compound CC1=CC2=C(C=C1C)NC(=O)CC2C3=CC=CC=C3OC completed. I found the following CIDs in PubChem with XLogP in the range of [4.3486695]+- 10%: [(2906028, 4), (2917087, 4.5), (17285549, 4.1)]
Predicted LogP value for compound COC1=CC=C2C(=C1)OC(CC2=O)(C(F)(F)F)O: [2.34140919]
Searching for similar compunds...
Filtering logP...
Request for compound COC1=CC=C2C(=C1)OC(CC2=O)(C(F)(F)F)O completed. I found the following CIDs in PubChem with XLogP in the range of [2.34140919]+- 10%: [(2917555, 2.4), (2849054, 2.2), (799275, 2.4)]
